In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,14 mins 47 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_l1ai0q
H2O cluster total nodes:,1
H2O cluster free memory:,1.876 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [2]:
train_high = h2o.import_file('../cleaned-data/03-train_high.csv')
train_low = h2o.import_file('../cleaned-data/03-train_low.csv')
test_high = h2o.import_file('../cleaned-data/03-test_high.csv')
test_low = h2o.import_file('../cleaned-data/03-test_low.csv')      

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 8)
(38730, 8)
(521, 7)
(12928, 7)


[None, None, None, None]

In [3]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat', 'cut', 'color', 'clarity', 'depth', 'table']

In [11]:
train_frame = train_high
model_name = 'diamonds_high'

aml_high = H2OAutoML(max_runtime_secs = 10, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |
11:24:10.895: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 4 models.
11:24:21.918: StackedEnsemble_AllModels_AutoML_20200301_112410 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
11:30:06.528: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 7 models.

████████████████████████████████████████████████████████| 100%

11:30:17.542: StackedEnsemble_AllModels_AutoML_20200301_113006 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted

predict
16609.7
16633.7
15779.9
17180.2
15962.5
13727.6
17050.4
16109.1
14724.2
15534.1


In [14]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [18]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = 10, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |
11:33:24.214: New models will be added to existing leaderboard diamonds_low@@price (leaderboard frame=null) with already 3 models.

████████████████████████████████████████████████████████| 100%

11:33:36.232: StackedEnsemble_AllModels_AutoML_20200301_113324 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
4833.32
1822.31
6899.89
1950.01
2780.32
1809.71
1425.89
3690.79
9856.66
2982.32


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200301_113230,514411,717.224,514411,438.472,nan
XGBoost_1_AutoML_20200301_113230,2.29288e+06,1514.23,2.29288e+06,879.278,0.297593
XGBoost_1_AutoML_20200301_113324,2.4297e+06,1558.75,2.4297e+06,901.379,0.331942
XGBoost_2_AutoML_20200301_113230,1.4155e+07,3762.31,1.4155e+07,2502.22,1.10923
XGBoost_2_AutoML_20200301_113324,1.57117e+07,3963.8,1.57117e+07,2601.98,1.19613


In [19]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [23]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [32]:
h2o.export_file(frame=combinedTable,path='../predictions/9-Auto-ML-Split-data-v2.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
